For this assignment you will be implementing your own verision of the perceptron algorithm.  Similar to the Naive Bayes assignment your Percptron class should be built to be compatible with the sklearn framework.  You can use the sklearn library and other python libraries to help with your implementation but you must implement the actual percpetron algorithm using your own code.

In [ ]:
#TODO: put your perceptron implemenation here

Split the diabetes data into and 80/20 train/test split.


In [ ]:
#TODO: split the diabetes data into training/test data

Using the diabetes training data with k-fold cross-validation (you choose the value for k) plot your perceptron's cross-validation accuracy and MCC as a function of the number of training epochs.  Use a learning rate of 0.10.

In [ ]:
#TODO: k-fold cross validation with a learning rate of 0.10

Choose a normalization/standardization strategy to apply to the diabetes training data. Using k-fold cross validation and plot the cross validation accuracy and MCC as a function of the number of training epochs.  Repeat for learning rates of 0.90, 0.50, 0.10, 0.01, and 0.0001.

In [ ]:
#TODO: k-fold cross validation on normalized/standardized data

How does the normalization/standardization affect the accuracy and MCC scores and number of training epochs?

Answer in this text box

On the scaled data, for each learning rate, how many training epochs are needed to acheive maximize the accuracy and MCC scores?

Answer in this text box

Is there a correlation between learning rates and the number of training epochs needed to acheive the optimal results?  If so, what is the correlation?  If not, why not?

Answer in this text box